# Miniproject 3 : Piecewise Regression and Resampling Techniques
   
-Dataset Ames Housing Dataset

This mini-project involves:    
- Piecewise Linear and Piecewise Polynomial Regression    
- Resampling Techniques: k-fold cross validation and bootstrapping

Submission: 

For this project, you will submit your .ipynb file through Blackboard. 

### Task 1: Piecewise Regression

For the Housing Dataset from mini-project1 :   

a. Randomly split the dataset into 80% training set and 20% validation set. (Feel free to write your own function to split data or use scikit learn functions)  
b. Perform:   
i)Multiple Linear Regression,  ii)Polynomial Regression of degree 2, iii)Piecwise Linear Regression (Split/cut K = 5, i.e. 6 bins), and iv) Spline with d = 3 and K = 3. 
          
Report:  
1. Epochs vs Training Error curve for all 4 models in b.    
2. Validation error for all 4 models.  Which one has the best Training and Validation Error?

Note :   

- Make sure you use all 5 input variables from Housing Data and same training and validation set from a) for all models in b.     
- You can still use the LRGD class you created in project 2 or use scikit learn. You just need to create new varaibles using appropriate indicator functions , express it as a additive model and use LRGD or use scikit learn.   
- Refer to general regression equation form for each models ii) to iv) on how to create new variables. 

Food for Thought:   
Q. Why we can still use LRGD we created on Task 1 for Polynomial, Piecewise and Splines? If you think, we cannot use the same Gradient Descent from LRGD, why not?

    



Before we do anything we first import SLRGD, but call it GLRGD, the generi linear regression with gradient descent, as we will be using it in a lot of contexts.

In [10]:
class GLRGD:
    def __init__(self, eta=0.01, n_iter=30, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state
# A member function that fits input X to output y using GD
    def fit(self, X, y):
        rgen = np.random.RandomState(self.random_state)
        self.beta = rgen.normal(loc=0.0, scale=0.01, size=X.shape[1])
        self.intercept = np.array([0.])
        self.losses = []

        for i in range(self.n_iter):
            output = self.net_input(X)
            errors = y - output
            # Update parameters of Linear Regression
            self.beta+= self.eta * 2.0 * X.T.dot(errors) / X.shape[0]
            self.intercept += self.eta * 2.0 * errors.mean()
            
            loss = (errors**2).mean()
            self.losses.append(loss)
        return self

    def net_input(self, X):
        return np.dot(X, self.beta) + self.intercept

    def predict(self, X):
        return self.net_input(X)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

columns = ['Overall Qual', 'Overall Cond', 'Gr Liv Area',
           'Central Air', 'Total Bsmt SF', 'SalePrice']

df = pd.read_csv('http://jse.amstat.org/v19n3/decock/AmesHousing.txt', 
                 sep='\t',
                 usecols=columns)
df.head()  

,Overall Qual,Overall Cond,Total Bsmt SF,Central Air,Gr Liv Area,SalePrice
0,6,5,1080.0,Y,1656,215000
1,5,6,882.0,Y,896,105000
2,6,6,1329.0,Y,1329,172000
3,7,5,2110.0,Y,2110,244000
4,5,5,928.0,Y,1629,189900


In [2]:
df = df.dropna(axis=0)
df['Central Air'] = df['Central Air'].map({'N': 0, 'Y': 1})
df.head()

,Overall Qual,Overall Cond,Total Bsmt SF,Central Air,Gr Liv Area,SalePrice
0,6,5,1080.0,1,1656,215000
1,5,6,882.0,1,896,105000
2,6,6,1329.0,1,1329,172000
3,7,5,2110.0,1,2110,244000
4,5,5,928.0,1,1629,189900


In [3]:
X = df[["Overall Qual","Overall Cond", "Total Bsmt SF","Central Air","Gr Liv Area"]]
y = df["SalePrice"]
print(X,y)

      Overall Qual  Overall Cond  Total Bsmt SF  Central Air  Gr Liv Area
0                6             5         1080.0            1         1656
1                5             6          882.0            1          896
2                6             6         1329.0            1         1329
3                7             5         2110.0            1         2110
4                5             5          928.0            1         1629
...            ...           ...            ...          ...          ...
2925             6             6         1003.0            1         1003
2926             5             5          864.0            1          902
2927             5             5          912.0            1          970
2928             5             5         1389.0            1         1389
2929             7             5          996.0            1         2000

[2929 rows x 5 columns] 0       215000
1       105000
2       172000
3       244000
4       189900
         ...

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

#### Multiple Regression

In [5]:
from sklearn.linear_model import LinearRegression


linreg = LinearRegression().fit(X_train,y_train)
y_pred = linreg.predict(X_test)

In [6]:
from sklearn import metrics

print(metrics.mean_squared_error(y_test,y_pred))

1441543342.977816


#### Polynomial Regression of degree 2

In [7]:
from sklearn.preprocessing import PolynomialFeatures
 
poly_features = PolynomialFeatures(degree = 2)
X_train_poly = poly_features.fit_transform(X_train)

linreg = LinearRegression()
linreg.fit(X_train_poly, y_train)



LinearRegression()

In [8]:
X_test_poly = poly_features.fit_transform(X_test)
y_pred_poly = linreg.predict(X_test_poly)
# print(metrics.mean_squared_error(y_test,y_pred))


In [9]:
print(metrics.mean_squared_error(y_pred_poly,y_test))

901721593.4690629


### Piecewise-linear regression

### Task 2: Ridge Regression and LASSO

1. For Housing Data, perform Ridge Regression and LASSO for the training data set from Task 1 for different values of lambda.   
    
    Report:    
    1. Values of lambda vs training error curve. 
    2. Values of lambda vs validation error. 
    3. Compare Validarion Error from MLR in Task 1 to Validation Error from Ridge and LASSO. 

    Do not forget to standarize all inputs. For Ridge and Lasso, we have not implemented our own solver. Please use scikit learn library. 
    

### Task 3: Resampling

For the Housing Data: (Complete Data set, not only training from Task 1)

1. Cross Validation: Perform Multiple Linear Regression with K-fold cross validation. You can choose K = 5.    
Report: 
    a. Regression Coeffs for each dataset. 
    b. Cross validation Error. 

2. Bootstrapping: Create 10 different datasets each with 3N/4 data samples using bootstrapping (i.e. Resampling with replacement), where N = total numbers of data samples in Housing Data. Perform Multiple Linear Regression on 10 datasets you created.

Report:   

a. Regression Coeffs for each dataset.  
b. Average error. 

You can write your own resample fucntions but feel free to use scikit learn resample. 



